# Setup a computer with AiiDA 

In [ ]:
import urllib.parse as urlparse
import ipywidgets as ipw
from traitlets import link
from aiidalab_widgets_base import SshComputerSetup, valid_sshcomputer_args
from aiidalab_widgets_base import AiidaComputerSetup, valid_aiidacomputer_args

from aiida.cmdline.commands import cmd_computer, cmd_code
import click
from click.testing import CliRunner
from glob import glob
import os
from jinja2 import Template

In [ ]:
parsed_url = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)
computer_list = {'fidis-che609': 'fidis.epfl.ch', 'deneb-che609':'deneb1.epfl.ch'}
THIS_DIR = os.path.dirname(os.path.realpath('aiidalab-lsmo'))
SETUP_DIR = os.path.join(THIS_DIR, 'setup')

select_computer = ipw.Dropdown(
    options=computer_list,
    label='fidis-che609',
    description='Select computer:',
    style = {'description_width': 'initial'}
)
display(select_computer)

## Step 1: Setup ssh
Note: The password is used only to set up the ssh connection and is never stored.

In [ ]:
args =  valid_sshcomputer_args(parsed_url)
sshcomputer = SshComputerSetup(**args)
link((select_computer, 'value'), (sshcomputer._inp_computer_hostname, 'value' ))
sshcomputer._use_proxy.value = True
sshcomputer._inp_proxy_address.value = 'lsmosrv1.epfl.ch'
display(sshcomputer)

## Step 2: Setup codes

In [ ]:
def render(template_file, **kwargs):
    """Produce yaml file from template.

    :param template_file: name of template file
    :param kwargs: will be forwarded to jinja2.Template
    """
    with open(template_file, 'r') as handle:
        content = handle.read()

    template = Template(content)
    yml_content = template.render(**kwargs)
    yml_file = os.path.splitext(template_file)[0]+'.yml'
    with open(yml_file, 'w') as handle:
        handle.write(yml_content)

def print_success(result):
    if not result.exit_code == 0:
        click.secho(result.output, fg='red')
    else:
        click.secho('Sucess', fg='green')

def setup(_=None):
    """Set up a given computer & codes."""
    computer = select_computer.label
    cli_runner = CliRunner()

    # computer setup
    computer_yml = SETUP_DIR + '/{c}/{c}.yml'.format(c=computer)
    username = sshcomputer.username    
    ssh_key ="{}/.ssh/id_rsa".format(os.path.expanduser("~"))
    options = ['--config', computer_yml]
    
    print("Setting up {}".format(computer))
    options = ['--config', computer_yml]
    result = cli_runner.invoke(cmd_computer.computer_setup, options)

    print("Configuring {}".format(computer))
    options = ['ssh', computer, '--username', username, '--safe-interval', 10, '--look-for-keys',
                   '--key-policy', 'AutoAddPolicy', '--non-interactive', '--key-filename', ssh_key]
    result = cli_runner.invoke(cmd_computer.computer_configure, options)
    print_success(result)

    for code_yml in glob(SETUP_DIR + '/{c}/*@*'.format(c=computer)):
        # code setup
        print("Setting up {}".format(code_yml))
        options = ['--config', code_yml]
        result = cli_runner.invoke(cmd_code.setup_code, options)
        print_success(result)

In [ ]:
setup_button = ipw.Button(description="Setup codes")
setup_button.on_click(setup)
display(ipw.HBox([setup_button]))